In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
orders_schema = "order_id long , order_date string, customer_id long,order_status string"

In [3]:
orders_new_df = spark.read \
.format("csv") \
.schema(orders_schema) \
.load("/public/trendytech/orders/orders_1gb.csv")

In [4]:
mapping_schema = "status string, code int"

In [5]:
mapping_df = spark.read \
.format("csv") \
.option("delimiter","|") \
.schema(mapping_schema) \
.load("/public/trendytech/datasets/mapping_data")

In [6]:
spark.conf.set('spark.sql.autoBroadcastJoinThreshold','-1')

In [7]:
orders_new_df.join(mapping_df, orders_new_df.order_status == mapping_df.status, "inner").write.format("noop").mode("overwrite").save()

In [8]:
spark.conf.set("spark.sql.adaptive.enabled", True)

In [9]:
orders_new_df.join(mapping_df, orders_new_df.order_status == mapping_df.status, "inner").write.format("noop").mode("overwrite").save()